### Tensor shape 변경
- reshape(), view() : 원소 갯수가 유지됨! 기존 텐서 공유함

In [30]:
import torch

In [31]:
# 텐서 데이터 생성
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(t1.shape, t1.ndim, t1.storage_offset())

torch.Size([2, 3]) 2


In [32]:
# [2,3] -> [3,2]로 형태 변경 : 원소는 동일하게 6개
t1.view(3,2)    # 1차원으로 쭉 읽어가면서 새로 나누기

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [33]:
# [-1, ] : -1 이면 상관 안하겠단 얘기, 알아서 지정하란 뜻
t1.view(-1, 1), t1.view(1, -1)

# 오류 1: RuntimeError: shape '[4, -1]' is invalid for input of size 6
# t1.view(4, -1) : 약수가 아니면 오류

# 오류 2 : RuntimeError: only one dimension can be inferred
# t1.view(-1, -1) : 1차원?

(tensor([[1],
         [2],
         [3],
         [4],
         [5],
         [6]]),
 tensor([[1, 2, 3, 4, 5, 6]]))

In [34]:
# 행렬 전환 : .T
print(t1.shape)
t2 = t1.T
print(t2.shape)

torch.Size([2, 3])
torch.Size([3, 2])


<hr> 

.reshape

In [35]:
# .view(-1, 6) : RuntimeError: view size is not compatible with input tensor's size and stride
# t2.view(-1, 6)

In [36]:
t2.reshape(-1, 6)   # .reshape은 가능

tensor([[1, 4, 2, 5, 3, 6]])

In [37]:
# 이유: .is_contiguous() -> 
t2.is_contiguous()  # False
t2.reshape(-1,6).is_contiguous()    # True

# 데이터 구조 차이 : ndarray는 메타 데이터를 포함한다; 아래부터 설명 시작

True

### 텐서 데이터의 메모리 저장 정보, 즉 메타데이터
- 현재 저장 형태, 검색 방향 정보, 시작 정보

In [38]:
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(t1.shape, t1.ndim)
print(f't1.storage() :\n{t1.storage()}')    # 데이터는 이렇게 1 어레이로 저장
#  1
#  2
#  3
#  4
#  5
#  6
# [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]

print(f't1.storage_offset() : {t1.storage_offset()}')
# t1.storage_offset() : 0   

print(f't1.stride() : {t1.stride()}')
# t1.stride() : (3, 1) => 행으로 3칸, 열로 1칸
# 1 2 3
# 4 5 6

# => 어디까지의 기준이지?

torch.Size([2, 3]) 2
t1.storage() :
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
t1.storage_offset() : 0
t1.stride() : (3, 1)


In [39]:
t2 = t1.view(-1, 2)
# 1 4
# 2 5
# 3 6

print(t2.shape, t2.ndim)
print(f't2.storage() :\n{t2.storage()}')
#  1
#  2
#  3
#  4
#  5
#  6
# [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]

print(f't2.storage_offset() : {t2.storage_offset()}')
# t2.storage_offset() : 0   

print(f't2.stride() : {t2.stride()}')   
# t2.stride() : (2, 1)

torch.Size([3, 2]) 2
t2.storage() :
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
t2.storage_offset() : 0
t2.stride() : (2, 1)


### 차원 제거/추가
- tensor.squeeze() : 텐서에서 차원이 1인 것을 제거
- tensor.unsqueeze(dim) : 텐서에서 차원을 추가

In [40]:
# 데이터 추가
t1 = torch.tensor([[1, 2], [3, 4]])
t2 = torch.tensor([[1, 2, 3, 4]])
t3 = torch.tensor([[[1, 2, 3, 4]]])

print(f'{t1.shape}, {t1.ndim}D, {t1.data_ptr()}')
print(f'{t2.shape}, {t2.ndim}D, {t2.data_ptr()}')
print(f'{t3.shape}, {t3.ndim}D, {t3.data_ptr()}')

torch.Size([2, 2]), 2D, 5177239208256
torch.Size([1, 4]), 2D, 5177239208320
torch.Size([1, 1, 4]), 3D, 5177239208448


In [41]:
t11 = t1.squeeze()
t22 = t2.squeeze()
t33 = t3.squeeze()
t44 = t3.unsqueeze(0)

print(f't1 차원 축소 : {t11.shape}, {t11.ndim}D, {t11.data_ptr()}')   # 축소할 차원이 없음
print(f't2 차원 축소 : {t22.shape}, {t22.ndim}D, {t22.data_ptr()}')   # 차원 축소됨
print(f't3 차원 축소 : {t33.shape}, {t33.ndim}D, {t33.data_ptr()}')   # 차원은 최대한으로 축소 : 3-> 1D
print(f't4 차원 추가 : {t44.shape}, {t44.ndim}D, {t44.data_ptr()}')   # 차원 추가, 주소는 같다.

t1 차원 축소 : torch.Size([2, 2]), 2D, 5177239208256
t2 차원 축소 : torch.Size([4]), 1D, 5177239208320
t3 차원 축소 : torch.Size([4]), 1D, 5177239208448
t4 차원 추가 : torch.Size([1, 1, 1, 4]), 4D, 5177239208448


원소/요소 수 변경 없이 1차원 증가시키기 : torch.unsqueeze(dim)

In [42]:
print(f't1 정보 : {t1.shape}, {t1.ndim}D, {t1.data_ptr()}, {t1.stride()}')

t1 정보 : torch.Size([2, 2]), 2D, 5177239208256, (2, 1)


In [43]:
t11 = t1.unsqueeze(1)
print(f't11 정보 : {t11.shape}, {t11.ndim}D, {t11.data_ptr()}, {t11.stride()}') # 1열에 차원 추가
t11 # 데이터는 들어있지않음

t11 정보 : torch.Size([2, 1, 2]), 3D, 5177239208256, (2, 2, 1)


tensor([[[1, 2]],

        [[3, 4]]])

In [44]:
t0 = t1.unsqueeze(dim=1)
t0

tensor([[[1, 2]],

        [[3, 4]]])

#### Tensor 차원/형태 변경

In [45]:
t1 = torch.tensor([[[1,2], [11, 22], [44, 55]]])
print(f't1 : {t1.shape}, {t1.ndim}D')

t1 : torch.Size([1, 3, 2]), 3D


In [46]:
# 2개의 차원을 변경하는 메서드
t11 = t1.transpose(0, 2)        # 0번에 2번을 넣겠다
print(f't11 : {t11.shape}, {t11.ndim}D')

# 다차원에서 .transpose는 교환할 차원을 선택해야함 (2차원은 자동)

t11 : torch.Size([2, 3, 1]), 3D


In [47]:
# 다 섞는 친구 .permute
t22 = t1.permute(1, 2, 0)       # 해당 인덱스를 넣음 : 첫번째에 인덱스1(=3), 두 번째에 인덱스2(=2), 세 번째에 인덱스0(=1)
print(f't22 : {t22.shape}, {t22.ndim}D')

t22 : torch.Size([3, 2, 1]), 3D


In [48]:
# 차원 추가 : None
t1.shape, t1[None].shape

(torch.Size([1, 3, 2]), torch.Size([1, 1, 3, 2]))

In [ ]:
# 일단 fin